# Identifying the disease

In [1]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download -d itachi9604/disease-symptom-description-dataset

!unzip disease-symptom-description-dataset

  0% 0.00/30.1k [00:00<?, ?B/s]
100% 30.1k/30.1k [00:00<00:00, 25.4MB/s]
Archive:  disease-symptom-description-dataset.zip
  inflating: Symptom-severity.csv    
  inflating: dataset.csv             
  inflating: symptom_Description.csv  
  inflating: symptom_precaution.csv  


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras import losses, optimizers, models, layers, callbacks
from sklearn.preprocessing import label_binarize

In [4]:
df = pd.read_csv('/content/dataset.csv')
df

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4916,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4917,Urinary tract infection,burning_micturition,bladder_discomfort,foul_smell_of urine,continuous_feel_of_urine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4918,Psoriasis,skin_rash,joint_pain,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
diseases = ['Acne','Common Cold','Diabetes ','Heart attack','Hyperthyroidism','Hypothyroidism','Malaria','Pneumonia']

#diseases = df['Disease'].unique().tolist()

In [6]:
df = df[df['Disease'].isin(diseases)]
df = df.drop_duplicates(ignore_index=True)
df['Disease'].value_counts()

Pneumonia          9
Common Cold        9
Hyperthyroidism    9
Diabetes           9
Hypothyroidism     8
Malaria            8
Acne               5
Heart attack       5
Name: Disease, dtype: int64

In [7]:
symptoms = []
for i in range(1, 18):
    symptoms.extend(df['Symptom_'+str(i)].values.tolist())

symptoms = list(set(symptoms))
symptoms.remove(np.nan)

symptoms = [x.strip() for x in symptoms]

len(symptoms), symptoms

(49,
 ['cold_hands_and_feets',
  'skin_rash',
  'pus_filled_pimples',
  'dizziness',
  'chest_pain',
  'excessive_hunger',
  'malaise',
  'headache',
  'blackheads',
  'lethargy',
  'high_fever',
  'weight_loss',
  'scurring',
  'muscle_weakness',
  'enlarged_thyroid',
  'obesity',
  'increased_appetite',
  'puffy_face_and_eyes',
  'fatigue',
  'sinus_pressure',
  'swelled_lymph_nodes',
  'runny_nose',
  'diarrhoea',
  'throat_irritation',
  'continuous_sneezing',
  'irritability',
  'vomiting',
  'loss_of_smell',
  'muscle_pain',
  'swollen_extremeties',
  'abnormal_menstruation',
  'depression',
  'redness_of_eyes',
  'rusty_sputum',
  'nausea',
  'phlegm',
  'congestion',
  'fast_heart_rate',
  'blurred_and_distorted_vision',
  'restlessness',
  'irregular_sugar_level',
  'cough',
  'brittle_nails',
  'weight_gain',
  'polyuria',
  'chills',
  'sweating',
  'mood_swings',
  'breathlessness'])

In [8]:
X = np.empty((0, len(symptoms)))
y = np.empty((0, len(diseases)))

for i in range(len(df)):
    x = np.zeros((1, len(symptoms)))
    y = np.append(y, label_binarize([diseases.index(df.iloc[i,0])], classes=range(len(diseases))), axis=0)
    for s in df.iloc[i,:].values[1:]:
        if isinstance(s, str):
            x[0,symptoms.index(s.strip())] = 1
    X = np.append(X, x, axis = 0)
    #print(X, y)

print(X.shape, y.shape)

(62, 49) (62, 8)


In [9]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, random_state = 13)

In [10]:
model = models.Sequential([
    layers.Dense(16, activation='relu', input_shape=Xtrain[0].shape),
    layers.Dense(8, activation='relu'),
    layers.Dense(len(diseases), activation='softmax')
])

cb = [callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

In [11]:
model.compile(loss=losses.CategoricalCrossentropy(),
              optimizer=optimizers.Adam(0.01),
              metrics=['accuracy'])

model.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), epochs=256, callbacks=cb)

Epoch 1/256
2/2 [==============================] - 1s 467ms/step - loss: 2.2117 - accuracy: 0.1713 - val_loss: 1.7535 - val_accuracy: 0.4615
Epoch 2/256
2/2 [==============================] - 0s 39ms/step - loss: 1.9348 - accuracy: 0.2570 - val_loss: 1.4886 - val_accuracy: 0.6154
Epoch 3/256
2/2 [==============================] - 0s 40ms/step - loss: 1.7072 - accuracy: 0.3459 - val_loss: 1.2630 - val_accuracy: 0.6923
Epoch 4/256
2/2 [==============================] - 0s 42ms/step - loss: 1.5445 - accuracy: 0.3699 - val_loss: 1.0594 - val_accuracy: 0.6923
Epoch 5/256
2/2 [==============================] - 0s 46ms/step - loss: 1.4094 - accuracy: 0.3803 - val_loss: 0.8828 - val_accuracy: 0.6923
Epoch 6/256
2/2 [==============================] - 0s 42ms/step - loss: 1.2852 - accuracy: 0.3595 - val_loss: 0.7338 - val_accuracy: 0.7692
Epoch 7/256
2/2 [==============================] - 0s 42ms/step - loss: 1.2004 - accuracy: 0.3971 - val_loss: 0.6186 - val_accuracy: 0.8462
Epoch 8/256
2/2 [==

In [12]:
model.evaluate(X, y)

2/2 [==============================] - 0s 3ms/step - loss: 0.1139 - accuracy: 1.0000


[0.11392734944820404, 1.0]

In [13]:
model.save('disease.h5')